In [1]:
# Installing libraries: 
# pip install geopy 

In [2]:
# 1
# removing duplciate lane entries
# removing lane info "_lane" from dataset
import pandas as pd

def clean_dataset(df):
    # Group the data by the two columns and keep only the first occurrence of each group
    reduced_df = df.drop_duplicates(subset=[5, 6], keep='first')
    # Convert column 1 values to strings and remove anything after and including the last underscore
    #reduced_df[1] = reduced_df[1].astype(str).str.rsplit('_', n=1).str[0]
    reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]

    reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
    reduced_df.iloc[:, 6] = reduced_df.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))

    return reduced_df


In [3]:
# 2
# group data based on sensor location description 
import pandas as pd

def get_sensor_group(data, sensor_loc, ignore=None):
    # Filter the DataFrame based on column 3 and ignore string
    mask = data[3].str.startswith(sensor_loc) & (~data[3].str.contains(ignore) if ignore is not None else True)
    matching_group = data[mask]
    return matching_group


In [4]:
import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def tsp(sensor_data):
    distances = calculate_distance(sensor_data)
    
    # Create an empty list to store the adjacency list
    adjacency_list = []

    # Create the folium map
    map = folium.Map(location=[sensor_data[5].mean(), sensor_data[6].mean()], zoom_start=10)

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors and add to adjacency list
    if min_i != -1 and min_j != -1:
        sensor1 = sensor_data.iloc[min_i]
        sensor2 = sensor_data.iloc[min_j]
        folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                        color='red').add_to(map)
        adjacency_list.append([sensor1[1], sensor2[1], min_distance])
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            sensor1 = sensor_data.iloc[min_i]
            sensor2 = sensor_data.iloc[min_j]
            folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                            color='red').add_to(map)
            adjacency_list.append([sensor1[1], sensor2[1], min_distance])
            connected.add(min_i)

    # Add sensor markers to the map
    for _, sensor in sensor_data.iterrows():
        tooltip_text = f"Sensor_ID: {sensor[1]}, Des: {sensor[3]}"
        folium.CircleMarker(
            location=[sensor[5], sensor[6]],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            tooltip=tooltip_text  # Add tooltip
            ).add_to(map)

    # Create a DataFrame from the adjacency list
    edge_list = pd.DataFrame(adjacency_list, columns=['sensor1', 'sensor2', 'cost'])
    
    return map, edge_list


In [6]:
# Main function

# Read from the original data and claen it:
data = clean_dataset(pd.read_csv('./detector/detectors2018.csv', header=None))

# Select sensor group:

#sensor_loc = "CC-215 WB"
#sensor_loc="CC-215 EB"
#sensor_loc="I-15 NB"
#sensor_loc= "I-515 SB"
#sensor_loc="US-95 SB"
sensor_loc="US-95 NB"


# Ignore "-" from the sensor group
ignore = 'Ramp'

# Get list of sensors using the sensor group
sensor_group = get_sensor_group(data, sensor_loc, ignore)

# Plot the map of the sensor group 

map, edge_list = tsp(sensor_group)
display(map)
print(edge_list)


      sensor1    sensor2        cost
0   421_3_507  452_1_508  398.821527
1   421_2_506  421_3_507  521.119646
2   421_1_505  421_2_506  543.986851
3   420_3_160  421_1_505  529.107786
4   420_2_160  420_3_160  519.764609
5   420_1_231  420_2_160  559.560881
6   419_3_217  420_1_231  575.378420
7   419_2_217  419_3_217  515.204372
8   419_1_234  419_2_217  537.507121
9   418_3_234  419_1_234  511.763187
10  418_2_225  418_3_234  490.235917
11  418_1_225  418_2_225  565.468593
12  417_3_230  418_1_225  563.803664
13  417_2_230  417_3_230  539.065230
14  417_1_216  417_2_230  541.163579
15  416_3_216  417_1_216  491.605855
16  416_2_224  416_3_216  508.964586
17  416_1_215  416_2_224  581.555789
18  415_3_215  416_1_215  582.886828
19  415_2_214  415_3_215  582.886316
20  415_1_223  415_2_214  522.311204
21  414_3_223  415_1_223  417.316073
22  414_2_209  414_3_223  418.640613
23  414_1_209  414_2_209  481.385717
24  413_3_204  414_1_209  522.964043
25  413_2_208  413_3_204  534.029685
2